In [132]:
from bugswarm.common.rest_api.database_api import DatabaseAPI
import requests
from tqdm import tqdm
bugswarmapi = DatabaseAPI(token="2vFV-ZCG70az8Fg84uNBvXw0ICnthMRvV83APgAjICY")
session = requests.Session()

In [133]:
def get_diff(repo: str, commit_sha: str):
    """
    Can be optimized
    """
    diff_code=[]
    added_code = []
    removed_code = []
    diff = session.get(f"https://github.com/{repo}/commit/{commit_sha}.diff").text
    line_counter=0
    file_counter=0
    for line in diff.split("\n"):
        if not line:
            continue
        line_counter +=1
        if len(line)>1 : 
            if line[0:2]=="++" or line_counter== len(diff.split("\n"))-1 :
                if file_counter!=0 :
                    diff_code.append({"added_code": added_code,
                              "removed_code": removed_code})
                    added_code = []
                    removed_code = []
                if line[0:2]=="++" : 
                    file_counter+=1 
            if line[0] == "+" and line[1] != "+":
                added_code.append(line[1:])  # remove +
            if line[0] == "-" and line[1]!= "-":
                removed_code.append(line[1:])
    print (file_counter) 
    return diff_code

In [134]:
test = get_diff("gwtbootstrap3/gwtbootstrap3", "41d93b14bf4c8ba2ae75d561ad3a640319e5a4ce")
for diff in test : 
    for file in test : 
        print (file['added_code'])

1
['<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"', '    xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">', '        <maven>3.2.1</maven>', '                <artifactId>maven-compiler-plugin</artifactId>', '                <version>${maven-compiler-plugin.version}</version>', '                    <source>${maven.compiler.source}</source>', '                    <target>${maven.compiler.target}</target>', '            <plugin>', '                <groupId>org.sonatype.plugins</groupId>', '                <artifactId>nexus-staging-maven-plugin</artifactId>', '                <version>${nexus-staging-maven-plugin.version}</version>', '                <extensions>true</extensions>', '                <configuration>', '                    <serverId>ossrh</serverId>', '                    <nexusUrl>https://oss.sonatype.org/</nexusUrl>', '                    <autoReleaseAfterClose>true<

In [131]:
#Applying a filter to treat java code 
api_filter = '{"lang":{"$in":["Java"]},"reproduce_successes":{"$gt":0},"build_system":{"$in":["Maven","Ant","Gradle"]},"classification.code":["Yes","Partial"]}'
bugswarmapi.filter_artifacts(api_filter)
#Building the dataset from BugSwarm to be similar to CC2Vec Dataset
dataset = [[],[],[],[]]
limit = 20
for i, artifact in tqdm(enumerate(bugswarmapi.list_artifacts())):
    for job, label in [("failed_job", 0), ("passed_job", 1)]:
        dataset[0].append(artifact[job]["trigger_sha"])
        diff = get_diff(artifact["repo"], artifact[job]["trigger_sha"])
        dataset[1].append(label)
        dataset[2].append("Trying to have the same dataset, but we don't care about this message")
        dataset[3].append(diff)
    if i > limit:
        break

1it [00:00,  1.89it/s]

2
1
2


2it [00:01,  1.90it/s]

2
3


3it [00:01,  1.95it/s]

...

1
1
1
22


21it [04:49,  2.04s/it]

9


21it [04:49, 13.80s/it]

1
2


In [135]:
import pickle
#Transform the dataset to a  jsonfile 
train_ids,train_labels,train_messages,train_code = dataset
#json_dataset = json.dumps(dataset)
#print(json_dataset[0][0]) 
#Convert the json format to a stripped file ( training data_set)
bugswarm_beta_training_dataset = open("bugswarm_beta2.0_training_dataset", "wb")
pickle.dump(dataset,bugswarm_beta_training_dataset)
bugswarm_beta_training_dataset.close() 

TypeError: list indices must be integers or slices, not str